In [1]:
import timeit
import h5py
import pandas as pd
import molml 
from molml.features import CoulombMatrix, BagOfBonds
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives


Using TensorFlow backend.


In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler 
from keras.models import Sequential
from keras import regularizers
from keras.layers.convolutional import Convolution1D
from keras.layers.core import Flatten, RepeatVector
from keras.layers.recurrent import GRU
from keras.layers.wrappers import TimeDistributed

In [ ]:
def vae_133(train, train_vali, epoch_no = 5, encoder_dim=[100, 100, 100], decoder_dim=[100, 100, 100]):
    
    # Variational autoencoder

    '''This script demonstrates how to build a variational autoencoder with Keras.
    Reference: "Auto-Encoding Variational Bayes" https://arxiv.org/abs/1312.6114
    The encoder_dim are the original dim, intermediate dim and latent dim respectivley
    
    '''   
    #batch_size = 100
    start = timeit.default_timer()

    x = Input(shape=(encoder_dim[0],))
    h = Dense(encoder_dim[1], activation='relu')(x)
    h = Dense(encoder_dim[1], activation='relu')(h)
    
    
    z_mean = Dense(encoder_dim[2])(h)
    z_log_var = Dense(encoder_dim[2])(h)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(encoder_dim[2], ), mean=0.)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(encoder_dim[2],))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later
    decoder_h = Dense( decoder_dim[1], activation='relu')
    
    decoder_mean = Dense(decoder_dim[0], activation='linear')
    
    h_decoded = decoder_h(z)
    
    x_decoded_mean = decoder_mean(h_decoded)

#Wa custom loss function: the sum of a reconstruction term, and the KL divergence regularization term.

    def vae_loss(x, x_decoded_mean):
        xent_loss = encoder_dim[0] * objectives.mean_absolute_error(x, x_decoded_mean)
#        xent_loss = encoder_dim[0] * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    vae = Model(x, x_decoded_mean)
    vae.compile(optimizer='rmsprop', loss=vae_loss, metrics=['mae', 'mse'])
    
    history = vae.fit(train, train,
            shuffle=True,
            nb_epoch=epoch_no,
            batch_size=100,
            verbose = 1,
           validation_data = (train_vali,train_vali))
    
    encoder = Model(x, z_mean)
    
    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60))
    
    return encoder, history

In [ ]:
def cnn_133(train, train_vali, epoch_no = 5, encoder_dim=[100, 100, 100], decoder_dim=[100, 100, 100]):
    
    # CNN autoencoder
   
    #batch_size = 100
    start = timeit.default_timer()
    x = Input(shape=(666,1 ))
    h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
    h = Flatten(name='flatten_1')(h)
    h = Dense(encoder_dim[2], activation = 'relu', name='dense_1')(h)
    
    
    z_mean = Dense(encoder_dim[2])(h)
    z_log_var = Dense(encoder_dim[2])(h)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(encoder_dim[2], ), mean=0.)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(encoder_dim[2],))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later
    decoder_h = Dense( decoder_dim[1], activation='relu')
    
    decoder_mean = Dense(decoder_dim[0], activation='linear')
    
    h_decoded = decoder_h(z)
    
    x_decoded_mean = decoder_mean(h_decoded)

#Wa custom loss function: the sum of a reconstruction term, and the KL divergence regularization term.

    def vae_loss(x, x_decoded_mean):
        xent_loss = encoder_dim[0] * objectives.mean_absolute_error(x, x_decoded_mean)
#        xent_loss = encoder_dim[0] * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

    vae = Model(x, x_decoded_mean)
    vae.compile(optimizer='rmsprop', loss=vae_loss, metrics=['mae', 'mse'])
    
    history = vae.fit(train, train,
            shuffle=True,
            nb_epoch=epoch_no,
            batch_size=100,
            verbose = 1,
           validation_data = (train_vali,train_vali))
    
    encoder = Model(x, z_mean)
    
    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60))
    
    return encoder, history

In [ ]:
vcnn_model = MoleculeVAE()
vcnn_model.create(latent_rep_size=10)
vcnn_model.autoencoder.fit(train_3D_, train_3D_,
                            shuffle=True,
                            nb_epoch=50,
                            batch_size=100,
                            verbose = 1)

#print ("The running takes %r min" %((stop-start)/60))

Epoch 1/50


In [22]:
class MoleculeVAE():

    autoencoder = None
    
    
    def create(self,
               max_length = 666,
               latent_rep_size = 2,
               weights_file = None):
        channel_length = 1
        
        x = Input(shape=(max_length, channel_length))
        _, z = self._buildEncoder(x, latent_rep_size, max_length)
        self.encoder = Model(x, z)

        encoded_input = Input(shape=(latent_rep_size,))
        self.decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                channel_length
            )
        )

        x1 = Input(shape=(max_length, channel_length ))
        vae_loss, z1 = self._buildEncoder(x1, latent_rep_size, max_length)
        self.autoencoder = Model(
            x1,
            self._buildDecoder(
                z1,
                latent_rep_size,
                max_length,
                channel_length
            )
        )

        if weights_file:
            self.autoencoder.load_weights(weights_file)
            self.encoder.load_weights(weights_file, by_name = True)
            self.decoder.load_weights(weights_file, by_name = True)

        self.autoencoder.compile(optimizer = 'Adam',
                                 loss = vae_loss,
                                 metrics = ['mae'])

    def _buildEncoder(self, x, latent_rep_size, max_length, epsilon_std = 0.01):
        h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
        h = Flatten(name='flatten_1')(h)
        h = Dense(20, activation = 'relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., std = epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.mean_absolute_error(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _buildDecoder(self, z, latent_rep_size, max_length, channel_length):
        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = Dense(10, name='gru_1')(h)
#        h = GRU(10, return_sequences = True, name='gru_1')(h)
        return TimeDistributed(Dense(channel_length, activation='linear'), name='decoded_mean')(h)

    def save(self, filename):
        self.autoencoder.save_weights(filename)
    
    def load(self, charset, weights_file, latent_rep_size = 292):
        self.create(charset, weights_file = weights_file, latent_rep_size = latent_rep_size)

In [19]:
# produce 3D vectors for training with CNN

train_3D_ = train_.reshape(train_.shape[0], train_.shape[-1],1)
train_vali_3D = train_vali.reshape(train_vali.shape[0], train_vali.shape[-1],1)

In [20]:
import keras
keras.__version__


'1.2.2'

In [ ]:
# build a baseline model to compare the performance between a vae encoded input and a raw input (666)
train_encoded, test_encoded = encoding(vcnn_model.encoder, train=train_3D_, test= train_vali_3D)

regressor = baseline_model(dim=[train_encoded.shape[1],200,1])
history_regress = regressor.fit(train_encoded, train_label,
        shuffle=True,
        nb_epoch=50,
        batch_size=100,
        verbose = 1
        )
regressor.evaluate(test_encoded, vali_label)


In [ ]:
input_dim = np.shape(train_)[1]

encoder, history = cnn_133(train_3D_, train_vali_3D, epoch_no = 20, \
                  encoder_dim=[input_dim, 666, 100], decoder_dim=[input_dim, 666, 100])

In [15]:
def baseline_vcnn(dim= [1,2,3]):
    
    #define a baseline model only consisting of one hidden layer
    
    latent_rep_size = 2
    batch_size = 100
    
    ##################

    x = Input(shape=(dim[-2], dim[-1]))
    h = Convolution1D(10, 11, activation = 'relu', name='conv_1')(x)
    h = Flatten(name='flatten_1')(h)
    h = Dense(10, activation = 'linear', name='dense_1')(h)
    
    z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
    z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)
    
    def sampling(args):
        z_mean_, z_log_var_ = args
        batch_size = K.shape(z_mean_)[0]
        epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0.)
        return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

    z = Lambda(sampling, output_shape=(latent_rep_size,))([z_mean, z_log_var])
    
    h_d = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
    h_d = RepeatVector(666, name='repeat_vector')(h_d)
#    h_d = GRU(10, return_sequences = True, name='gru_3')(h_d)
    
    h_d = TimeDistributed(Dense(dim[-1], activation='linear'), name='decoded_mean')(h_d)

    def vae_loss(x, x_decoded_mean):
        x = K.flatten(x)
        x_decoded_mean = K.flatten(x_decoded_mean)
        xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
        return xent_loss + kl_loss


    
    
    cnn = Model(x, h_d)

    cnn.compile(loss='mean_absolute_error', optimizer='adam')
    return cnn

In [ ]:
input_dim = np.shape(train_)[1]

vcnn_base = baseline_vcnn(dim=[input_dim, 666, 1])
vcnn_base.fit(train_3D_, train_label,
        shuffle=True,
        nb_epoch=50,
        batch_size=100,
        verbose = 1,
        )

In [ ]:
def baseline_cnn(dim= [1,2,3]):
    
    #define a baseline model only consisting of one hidden layer

    x = Input(shape=(dim[-2], dim[-1]))
    h = Convolution1D(10, 11, activation = 'relu', name='conv_1')(x)
    h = Flatten(name='flatten_1')(h)
    h = Dense(1, activation = 'linear', name='dense_1')(h)
    cnn = Model(x, h)

    cnn.compile(loss='mean_absolute_error', optimizer='adam')
    return cnn

In [ ]:
input_dim = np.shape(train_)[1]

cnn_base = baseline_cnn(dim=[input_dim, 666, 1])
cnn_base.fit(train_3D_, train_label,
        shuffle=True,
        nb_epoch=50,
        batch_size=100,
        verbose = 1,
        )

In [ ]:
input_dim = np.shape(train_)[1]

encoder, history = vae_133(train_, train_vali, epoch_no = 20, \
                  encoder_dim=[input_dim, 600, 200], decoder_dim=[input_dim, 600, 200])

In [ ]:
# build a baseline model to compare the performance between a vae encoded input and a raw input (666)
train_encoded, test_encoded = encoding(encoder, train=train, test= test)

regressor = baseline_model(dim=[train_encoded.shape[1],200,1])
history_regress = regressor.fit(train_encoded, trainlabel_12,
        shuffle=True,
        nb_epoch=50,
        batch_size=100,
        verbose = 0,
        )
regressor.evaluate(test_encoded, testlabel_12)

In [ ]:
def baseline_model(dim= [1,2,3]):
    
    #define a baseline model only consisting of one hidden layer
	# create model
	model = Sequential()
	model.add(Dense(dim[1], input_dim=dim[0],  activation='relu'))
	model.add(Dense(dim[-1], activation='linear'))
	# Compile model
	model.compile(loss='mean_absolute_error', optimizer='adam')
	return model

In [3]:
#load molecule data, the coordinate, atom_number, elements are essential as the input 

attrs_list =[ 'atom_number', 'data_base', 'atom_list', 'atom_coordinate_list', 'frequency_list','target_list',\
          'smile_list', 'atom_charge']

metrics = [r2_score, mean_absolute_error, mean_squared_error, median_absolute_error]

def load_133k(num = None):
    
    data_path = '/home/peng/Documents/Project_C/QSAR_nlp/Dataset_qm9/'
    data_set = "133K.hdf5"

    coord_list = []
    element_list = []
    target_list = []
    atom_no_list = []
    

    f = h5py.File(data_path + data_set, "r")
    if not num:
        limit = len(f)+1
    else:
        limit = num
        
    for i in range(1, limit):
        str_element = str(f[str(i)].attrs[attrs_list[2]], 'utf-8')
        split_str = []
        for j in str_element:
            split_str.append(j) 
        

        element_list.append(np.array(split_str))
#        element_list.append(f[str(i)].attrs[attrs_list[2]])
        coord_list.append(f[str(i)].attrs[attrs_list[3]])    
        target_list.append(f[str(i)].attrs[attrs_list[-3]]) 
        atom_no_list.append(f[str(i)].attrs[attrs_list[0]])   
        if i %100000 == 0:
            print ("already finish ", i)
    f.close()
    
    #df = pd.DataFrame({'coord':coord_list, 'element':element_list, 'target':target_list})
    
    return element_list, coord_list, target_list, atom_no_list

def get_attributes(element_list, coord_list, target_list):

    fit_list = []
#    target_single_list = []
    for i in range(0, len(element_list)):
        fit_list.append((element_list[i], coord_list[i]))
#        target_single_list.append(target_list[i][-1])
        
    return fit_list #, target_single_list

def get_train_list(num=None):

    # extract the element, the coordinates, the targets and the atom_no from the 133k dataset
    element_list, coord_list, target_list, atom_no_list = load_133k(num=num)
    # generate the (element, coordinate) tuple for following feature engineering
    fit_list = get_attributes(element_list, coord_list, target_list)

    # generate feature engineered input features, CM or BOB
    feature_methods = [CoulombMatrix(), BagOfBonds()]
    feat_co = feature_methods[1]
    train_list = feat_co.fit_transform(fit_list)
    return train_list



In [17]:
df = pd.read_csv('bob_targets15.csv', header = 0)

train_list = get_train_list()

print (np.shape(train_list))

already finish  100000
(133885, 666)


In [ ]:
trainlabel_12, testlabel_12 = train_test_split(df['12'], test_size = 0.2, random_state = 32) 

In [18]:
# apply minmax to train_list 

train_list_scale = MinMaxScaler().fit_transform(train_list)

target_single_list = df['7']

train, test, trainlabel, testlabel = train_test_split(train_list_scale, target_single_list,\
                                                      test_size=0.2, random_state = 32)

train_, train_vali, train_label, vali_label = train_test_split(train, trainlabel, \
                                                              test_size=0.2, random_state = 32)

In [ ]:
df = pd.read_csv('targets_15.csv', header = 0)

In [ ]:
df[['5','6','7','9', '10','11','12','13']].corr()

In [ ]:
np.shape(train_)

In [ ]:
print(history.history.keys())
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])


In [ ]:
# try to use DAE 
input_dim = np.shape(train_)[1]

encoder_dae, history_dae = dae_133(train_, train_vali, epoch_no = 20, \
                  encoder_dim=[input_dim, 600, 100], decoder_dim=[input_dim, 600, 100])

train_encoded_dae, test_encoded_dae = encoding(encoder_dae, train=train, test= test)

# build a baseline model to compare the performance between a vae encoded input and a raw input (666)
regressor_dae = baseline_model(dim=[100,200,1])

history_regress = regressor_dae.fit(train_encoded_dae, trainlabel,
        shuffle=True,
        nb_epoch=50,
        batch_size=100,
        verbose = 0,
        )
regressor_dae.evaluate(test_encoded_dae, testlabel)

In [ ]:
# build a model to project inputs on the latent space
def encoding(encoder= None, train = train, test = test):
    train_encoded = encoder.predict(train)
    #print(np.shape(train)[1])
    test_encoded = encoder.predict(test)
    #print(np.shape(test))
#    train_encoded_ = encoder.predict(train_)
    #print(np.shape(train)[1])
#    vali_encoded = encoder.predict(train_vali)
    encoded_scale= MinMaxScaler().fit(train_encoded)
    train_encoded_scale= encoded_scale.transform(train_encoded)
    test_encoded_scale= encoded_scale.transform(test_encoded)
    print 'the dimension for lattent representation is ', np.shape(train_encoded)[1]
    return train_encoded_scale, test_encoded_scale


In [ ]:
color_label_int = [str(round(item/np.max(trainlabel))) for item in trainlabel]

In [ ]:
from sklearn.decomposition import PCA
pca_fit = PCA(n_components=100).fit(train)

print (pca_fit.explained_variance_ratio_ )
print (np.sum(pca_fit.explained_variance_ratio_))
#print (np.shape(newtrain))

pca_train = pca_fit.transform(train)
pca_test = pca_fit.transform(test)

In [ ]:
def dae_133(train, train_vali, epoch_no = 5, encoder_dim=[100, 100, 100], decoder_dim=[100, 100, 100]):
    
    ### Denosing autoencoder

    noise_factor = 0.5
    
    train_noisy = train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train.shape) 


    train_noisy = np.clip(train_noisy, 0., 1.)


    # this is the size of our encoded representations
    encoding_dim = 6  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    input_dim= Input(shape=(encoder_dim[0],))
    # "encoded" is the encoded representation of the input
    encoded = Dense(encoder_dim[1], activation='relu')(input_dim)
    encoded = Dense(encoder_dim[-1], activation='relu', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)

    decoded = Dense(decoder_dim[1], activation='relu')(encoded)
    decoded = Dense(decoder_dim[0], activation='relu')(decoded)


    # this model maps an input to its reconstruction
    autoencoder = Model(input=input_dim, output=decoded)

    
    autoencoder.compile(optimizer='sgd', loss='mae')

    start = timeit.default_timer()
    history = autoencoder.fit(train, train,
                nb_epoch=epoch_no,
                batch_size=100,
                shuffle=True
                )
    encoder = Model(input=input_dim, output=encoded)

    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60))
    
    return encoder, history

In [ ]:
estimator = rfr().fit(train_encoded, trainlabel)
predict_test = estimator.predict(test_encoded)
for i in metrics:
    print i(testlabel, predict_test)

In [ ]:
estimator = rfr().fit(pca_train, trainlabel)
predict_test = estimator.predict(pca_test)
for i in metrics:
    print i(testlabel, predict_test)

In [ ]:
list_all = []
for i in range(0, len(element_list)):
    for j in element_list[i]:
        if j not in set(list_all):
            list_all.append(j)

In [ ]:
df_atom = pd.DataFrame({'atom_no':atom_no_list})
df_atom.describe()
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')
plt.figure()
df_atom.plot.hist(alpha = 0.5, bins = 15)
plt.show()

In [ ]:
import seaborn as sns
color_label = [str(item/np.max(train_label)) for item in train_label]
plt.figure(figsize=(15,12))
plt.scatter(train_encoded[:,0], train_encoded[:,1],marker = 'o', linewidth = '0', cmap = plt.cm.coolwarm,\
            c = color_label, s = 20, alpha = 1.0)
plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(15,12))
plt.scatter(pca_train[:,0], pca_train[:,1] ,marker = 'o', linewidth = '0', cmap = plt.cm.coolwarm,\
            c = color_label, s = 40)
plt.colorbar()
plt.show()

In [ ]:
'''
generate targets, bob and store them into bob_targets15.csv

# extract the element, the coordinates, the targets and the atom_no from the 133k dataset
element_list, coord_list, target_list, atom_no_list = load_133k()
# generate the (element, coordinate) tuple for following feature engineering
fit_list = get_attributes(element_list, coord_list, target_list)

# generate feature engineered input features, CM or BOB
feature_methods = [CoulombMatrix(), BagOfBonds()]
feat_co = feature_methods[1]
train_list = feat_co.fit_transform(fit_list)

df = pd.read_csv('targets_15.csv', header = 0)
df['bob'] = train_list.tolist()
df.to_csv('bob_targets15.csv', header = True)

'''